In [1]:
from google.colab import files

uploaded = files.upload()

Saving celulares_limpios.json to celulares_limpios.json
Saving intents.json to intents.json


In [13]:
from google.colab import files

uploaded = files.upload()

Saving IntentosXEficiencia.json to IntentosXEficiencia (1).json


codigo origen corriendo

In [7]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC  # 📌 Mejor clasificador que KNN

# 📌 Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"  # Modelo para obtener embeddings
modelo_resp_name = "facebook/blenderbot-400M-distill"  # Modelo para generar respuestas

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# 📌 Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# 📌 Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# 📌 Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])


if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

while True:
    pregunta = input("👤 Escribe tu consulta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        break
    print("🤖", responder_chatbot(pregunta))


👤 Escribe tu consulta (o 'salir' para terminar): ¿cuál es el mejor celular para gaming'
🤖 ¡Cuídate!
👤 Escribe tu consulta (o 'salir' para terminar): celular barato
🤖 Los teléfonos con Snapdragon 8 Gen 2 y buena tasa de refresco son ideales para juegos.
👤 Escribe tu consulta (o 'salir' para terminar): telefono para jugar
🤖 ¿Tienes un presupuesto en mente? Puedo recomendarte algo según tu rango de precio.
👤 Escribe tu consulta (o 'salir' para terminar): mil dolares
🤖 Para gaming, te recomiendo un celular con un buen procesador y al menos 8GB de RAM.
👤 Escribe tu consulta (o 'salir' para terminar): Conoces algun modelo en especifico?
🤖 Busca un celular con una GPU potente y refrigeración eficiente.
👤 Escribe tu consulta (o 'salir' para terminar): Recomienda un telefono
🤖 Los teléfonos con Snapdragon 8 Gen 2 y buena tasa de refresco son ideales para juegos.
👤 Escribe tu consulta (o 'salir' para terminar): Modelo
🤖 Busca un celular con una GPU potente y refrigeración eficiente.
👤 Escribe tu

EFICIENCIA

efi1 con preguntas y respestas aleatorias esperadas

In [8]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas]

    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

    print(f"Precisión (Accuracy): {accuracy:.2f}")
    print(f"Precisión (Precision): {precision:.2f}")

# Ejemplo de preguntas y respuestas para la evaluación
preguntas_evaluacion = [
    "¿Qué tipo de pantalla tiene el iPhone 13?",
    "¿Cuánto cuesta el Samsung Galaxy S22?",
    "¿Qué procesador tiene el Xiaomi Redmi Note 11?",
    "¿Cuántos megapíxeles tiene la cámara del Google Pixel 6?",
    "¿Tiene el OnePlus 10 Pro carga rápida?",
    "¿Qué versión de Android tiene el Samsung Galaxy A53?",
    "¿Es resistente al agua el iPhone 13 Pro?",
    "¿Tiene el Xiaomi 12 lector de huellas digitales?",
    "¿Qué celulares tienen 5G?",
    "¿Qué celulares tienen buena cámara para selfies?",
    "¿Cuál es el mejor celular para juegos?",
    "¿Qué celular tiene la mejor batería?",
    "¿Qué celular tiene la mejor cámara?",
    "¿Qué celular tiene la mejor pantalla?",
    "¿Qué celular es el más resistente?",
    "¿Qué celular tiene la mejor relación calidad-precio?",
    "¿Qué celular tiene la mejor conectividad?",
    "¿Qué celular tiene la mejor seguridad?",
    "¿Qué celular tiene la mejor experiencia de usuario?",
    "¿Qué celular tiene el mejor diseño?",
    "¿Qué celulares tienen pantalla AMOLED?",
    "¿Qué celulares tienen 120Hz?",
    "¿Qué celulares tienen carga inalámbrica?",
    "¿Qué celulares tienen zoom óptico?",
    "¿Qué celulares tienen altavoces estéreo?",
    "¿Qué celulares tienen jack de auriculares?",
    "¿Qué celulares tienen ranura para tarjeta microSD?",
    "¿Qué celulares tienen lápiz óptico?",
    "¿Qué celulares tienen pantalla plegable?",
    "¿Qué celulares tienen pantalla curva?",
    "¿Qué celulares tienen notch?",
    "¿Qué celulares tienen agujero en pantalla?",
    "¿Qué celulares tienen cámara frontal debajo de la pantalla?",
    "¿Qué celulares tienen cámara TOF?",
    "¿Qué celulares tienen sensor LiDAR?",
    "¿Qué celulares tienen sensor de ritmo cardíaco?",
    "¿Qué celulares tienen sensor de temperatura?",
    "¿Qué celulares tienen sensor de luz ultravioleta?",
    "¿Qué celulares tienen sensor de huellas dactilares ultrasónico?",
    "¿Qué celulares tienen sensor de huellas dactilares óptico?"
]

respuestas_esperadas = [
    "El iPhone 13 tiene una pantalla OLED Super Retina XDR de 6.1 pulgadas.",
    "El precio del Samsung Galaxy S22 varía según la configuración, pero ronda los $799.",
    "El Xiaomi Redmi Note 11 tiene un procesador Qualcomm Snapdragon 680.",
    "La cámara principal del Google Pixel 6 tiene 50 megapíxeles.",
    "Sí, el OnePlus 10 Pro tiene carga rápida SuperVOOC de 80W.",
    "El Samsung Galaxy A53 viene con Android 12.",
    "Sí, el iPhone 13 Pro es resistente al agua con certificación IP68.",
    "Sí, el Xiaomi 12 tiene lector de huellas digitales en pantalla.",
    "Algunos celulares con 5G son: iPhone 13, Samsung Galaxy S22, Xiaomi 12, Google Pixel 6, OnePlus 10 Pro.",
    "Algunos celulares con buena cámara para selfies son: Google Pixel 6, iPhone 13, Samsung Galaxy S22.",
    "El mejor celular para juegos depende de tus preferencias, pero algunos modelos populares son el Asus ROG Phone 6, el RedMagic 7S Pro y el iPhone 14 Pro Max.",
    "El celular con la mejor batería depende de tus necesidades, pero algunos modelos con buena autonomía son el Asus ROG Phone 6, el Motorola Edge+ y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor cámara depende de tus preferencias, pero algunos modelos con cámaras destacadas son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor pantalla depende de tus preferencias, pero algunos modelos con pantallas excelentes son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "El celular más resistente depende de tus necesidades, pero algunos modelos con buena resistencia son el CAT S62 Pro, el Samsung Galaxy XCover 6 Pro y el Doogee S100.",
    "El celular con la mejor relación calidad-precio depende de tus necesidades y presupuesto, pero algunos modelos populares son el Google Pixel 6a, el Samsung Galaxy A53 y el Xiaomi Redmi Note 11 Pro.",
    "El celular con la mejor conectividad depende de tus necesidades, pero algunos modelos con buena conectividad son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "El celular con la mejor seguridad depende de tus necesidades, pero algunos modelos con buena seguridad son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor experiencia de usuario depende de tus preferencias, pero algunos modelos con buena experiencia de usuario son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con el mejor diseño depende de tus preferencias, pero algunos modelos con diseños atractivos son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "Algunos celulares con pantalla AMOLED son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11.",
    "Algunos celulares con 120Hz son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con carga inalámbrica son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con zoom óptico son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, Xiaomi 13 Pro.",
    "Algunos celulares con altavoces estéreo son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con jack de auriculares son: Asus ROG Phone 6, Sony Xperia 1 IV, algunos modelos de gama de entrada y media.",
    "Algunos celulares con ranura para tarjeta microSD son: Algunos modelos de Samsung Galaxy A, algunos modelos de Xiaomi Redmi Note.",
    "Algunos celulares con lápiz óptico son: Samsung Galaxy S23 Ultra, Samsung Galaxy Note 20 Ultra.",
    "Algunos celulares con pantalla plegable son: Samsung Galaxy Z Fold 4, Samsung Galaxy Z Flip 4, Motorola Razr 2022.",
    "Algunos celulares con pantalla curva son: Samsung Galaxy S23 Ultra, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con notch son: iPhone 13, iPhone 12, algunos modelos de gama de entrada y media.",
    "Algunos celulares con agujero en pantalla son: Samsung Galaxy S23 Ultra, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con cámara frontal debajo de la pantalla son: Samsung Galaxy Z Fold 4, Xiaomi Mix 4.",
    "Algunos celulares con cámara TOF son: Algunos modelos de gama alta de Huawei, algunos modelos de gama alta de Samsung.",
    "Algunos celulares con sensor LiDAR son: iPhone 14 Pro Max, iPhone 13 Pro Max, iPad Pro.",
    "Algunos celulares con sensor de ritmo cardíaco son: Algunos modelos de Samsung Galaxy Watch.",
    "Algunos celulares con sensor de temperatura son: Algunos modelos de Huawei Watch.",
    "Algunos celulares con sensor de luz ultravioleta son: Algunos modelos de Samsung Galaxy S.",
    "Algunos celulares con sensor de huellas dactilares ultrasónico son: Samsung Galaxy S23 Ultra, Samsung Galaxy S22 Ultra.",
    "Algunos celulares con sensor de huellas dactilares óptico son: Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro."
]
evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas)


Precisión (Accuracy): 0.80
Precisión (Precision): 0.83


efi2 prueba de parametros del intents.json (la eficiencia es baja debido a que non tiene respuestas esperadas especificas)

In [11]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas]

    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

    print(f"Precisión (Accuracy): {accuracy:.2f}")
    print(f"Precisión (Precision): {precision:.2f}")
preguntas_evaluacion = [
    # Intención "saludo"
    "Hola",
    "¿Cómo estás?",
    "Buenos días",
    "¡Qué tal!",
    "Hey",
    "¿Qué onda?",
    "Dame saludos",
    "¿Me saludas?",

    # Intención "despedida"
    "Adiós",
    "Hasta luego",
    "Nos vemos",
    "Chao",
    "Me voy",
    "Bye",
    "Hasta la vista",
    "Cuídate",
    "Debo irme",
    "Despídete",

    # Intención "agradecimientos"
    "gracias",
    "muchas gracias",
    "mil gracias",
    "muy amable",
    "se lo agradezco",
    "fue de ayuda",
    "gracias por la ayuda",
    "muy agradecido",
    "gracias por tu tiempo",
    "gracias por responder",
    "Te agradezco mucho",

    # Intención "celulares_para_juegos"
    "jugar",
    "gaming",
    "potente para juegos",
    "rendimiento en juegos",
    "Teléfonos gaming",
    "Me gustan los juegos",
    "para jugar",
    "buena GPU",
    "mejor celular gamer",
    "¿Qué celular es bueno para jugar?",

    # Intención "cámara_y_fotografía"
    "buena cámara",
    "tomar fotos",
    "cámara de alta calidad",
    "mejor cámara",
    "fotografía",
    "fotos profesionales",
    "buen zoom",
    "cámara nocturna",
    "estabilización óptica",
    "cámara frontal",
    "¿Qué celular tiene buena cámara para fotos?",

    # Intención "resistencia_y_durabilidad"
    "resistente",
    "no se rompa fácil",
    "aguante golpes",
    "certificación IP68",
    "resistente al agua",
    "a prueba de polvo",
    "todoterreno",
    "reforzado",
    "Gorilla Glass",
    "¿Qué celular es resistente al agua y golpes?",

    # Intención "trabajo_y_productividad"
    "trabajo",
    "negocios",
    "multitarea",
    "S Pen",
    "almacenamiento",
    "rápido",
    "videollamadas",
    "productividad",
    "Microsoft Office",
    "batería",
    "¿Qué celular es bueno para trabajar?",

    # Intención "software_y_actualizaciones"
    "soporte de actualizaciones",
    "soporte de software",
    "seguridad actualizada",
    "Android 15",
    "Actualizacion",
    "¿Qué celular tiene la última versión de android?",

    # Intención "celulares_por_marca"
    "Samsung",
    "Apple",
    "Xiaomi",
    "Motorola",
    "Realme",
    "Huawei",
    "Google Pixel",
    "OnePlus",
    "Asus",
    "Sony",
    "¿Qué celulares tiene Samsung?",

    # Intención "pantalla_y_calidad"
    "pantalla",
    "AMOLED",
    "Hercios",
    "Hz",
    "pulgadas",
    "OLED",
    "HDR10",
    "resolución",
    "¿Qué celular tiene buena pantalla?",

    # Intención "celulares_nuevos_y_antiguos"
    "nuevo",
    "este año",
    "reciente",
    "última generación",
    "¿Qué celulares nuevos hay?"
]

respuestas_esperadas = [
    # Intención "saludo"
    "saludo",
    "saludo",
    "saludo",
    "saludo",
    "saludo",
    "saludo",
    "saludo",
    "saludo",

    # Intención "despedida"
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",
    "despedida",

    # Intención "agradecimientos"
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",
    "agradecimientos",

    # Intención "celulares_para_juegos"
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",
    "celulares_para_juegos",

    # Intención "cámara_y_fotografía"
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",
    "cámara_y_fotografía",

    # Intención "resistencia_y_durabilidad"
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",
    "resistencia_y_durabilidad",

    # Intención "trabajo_y_productividad"
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",
    "trabajo_y_productividad",

    # Intención "software_y_actualizaciones"
    "software_y_actualizaciones",
    "software_y_actualizaciones",
    "software_y_actualizaciones",
    "software_y_actualizaciones",
    "software_y_actualizaciones",
    "software_y_actualizaciones",

    # Intención "celulares_por_marca"
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",
    "celulares_por_marca",

    # Intención "pantalla_y_calidad"
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",
    "pantalla_y_calidad",

    # Intención "celulares_nuevos_y_antiguos"
    "celulares_nuevos_y_antiguos",
    "celulares_nuevos_y_antiguos",
    "celulares_nuevos_y_antiguos",
    "celulares_nuevos_y_antiguos",
    "celulares_nuevos_y_antiguos"
]
evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas)

Precisión (Accuracy): 0.22
Precisión (Precision): 0.23


In [16]:
!pip install transformers scikit-learn

PRUEBA 2.0 CON LOS INTENTS.JSON COMPLETOS

In [38]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas]

    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)


import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas if respuesta is not None]

    if y_true and y_pred:
        y_true_encoded = label_encoder.transform(y_true)
        y_pred_encoded = label_encoder.transform(y_pred)

        accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
        precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
        recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
        f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

        print(f"Precisión (Accuracy): {accuracy:.2f}")
        print(f"Precisión (Precision): {precision:.2f}")
    else:
        print("No se pudieron generar predicciones para calcular las métricas.")


preguntas_evaluacion = [

    # Intención "saludo"
    "Hola",
    "¿Cómo estás?",
    "Buenos días",
    "¡Qué tal!",
    "Hey",
    "¿Qué onda?",
    "Dame saludos",
    "¿Me saludas?",

    # Intención "despedida"
    "Adiós",
    "Hasta luego",
    "Nos vemos",
    "Chao",
    "Me voy",
    "Bye",
    "Hasta la vista",
    "Cuídate",
    "Debo irme",
    "Despídete",

    # Intención "agradecimientos"
    "gracias",
    "muchas gracias",
    "mil gracias",
    "muy amable",
    "se lo agradezco",
    "fue de ayuda",
    "gracias por la ayuda",
    "muy agradecido",
    "gracias por tu tiempo",
    "gracias por responder",
    "Te agradezco mucho",

    # Intención "celulares_para_juegos"
    "jugar",
    "gaming",
    "potente para juegos",
    "rendimiento en juegos",
    "Teléfonos gaming",
    "Me gustan los juegos",
    "para jugar",
    "buena GPU",
    "mejor celular gamer",
    "¿Qué celular es bueno para jugar?",

    # Intención "cámara_y_fotografia"
    "buena cámara",
    "tomar fotos",
    "cámara de alta calidad",
    "mejor cámara",
    "fotografía",
    "fotos profesionales",
    "buen zoom",
    "cámara nocturna",
    "estabilización óptica",
    "cámara frontal",
    "¿Qué celular tiene buena cámara para fotos?",

    # Intención "resistencia_y_durabilidad"
    "resistente",
    "no se rompa",
    "¿Cómo estás?",
    "Buenos días",
    "¡Qué tal!",
    "Hey",
    "¿Qué onda?",
    "Dame saludos",
    "¿Me saludas?",

    # Intención "despedida"
    "Adiós",
    "Hasta luego",
    "Nos vemos",
    "Chao",
    "Me voy",
    "Bye",
    "Hasta la vista",
    "Cuídate",
    "Debo irme",
    "Despídete",

    # Intención "agradecimientos"
    "gracias",
    "muchas gracias",
    "mil gracias",
    "muy amable",
    "se lo agradezco",
    "fue de ayuda",
    "gracias por la ayuda",
    "muy agradecido",
    "gracias por tu tiempo",
    "gracias por responder",
    "Te agradezco mucho",

    # Intención "celulares_para_juegos"
    "jugar",
    "gaming",
    "potente para juegos",
    "rendimiento en juegos",
    "Teléfonos gaming",
    "Me gustan los juegos",
    "para jugar",
    "buena GPU",
    "mejor celular gamer",
    "¿Qué celular es bueno para jugar?",

    # Intención "cámara_y_fotografía"
    "buena cámara",
    "tomar fotos",
    "cámara de alta calidad",
    "mejor cámara",
    "fotografía",
    "fotos profesionales",
    "buen zoom",
    "cámara nocturna",
    "estabilización óptica",
    "cámara frontal",
    "¿Qué celular tiene buena cámara para fotos?",

    # Intención "resistencia_y_durabilidad"
    "resistente",
    "no se rompa fácil",
    "aguante golpes",
    "certificación IP68",
    "resistente al agua",
    "a prueba de polvo",
    "todoterreno",
    "reforzado",
    "Gorilla Glass",
    "¿Qué celular es resistente al agua y golpes?",

    # Intención "trabajo_y_productividad"
    "trabajo",
    "negocios",
    "multitarea",
    "S Pen",
    "almacenamiento",
    "rápido",
    "videollamadas",
    "productividad",
    "Microsoft Office",
    "batería",
    "¿Qué celular es bueno para trabajar?",

    # Intención "software_y_actualizaciones"
    "soporte de actualizaciones",
    "soporte de software",
    "seguridad actualizada",
    "Android 15",
    "Actualizacion",
    "¿Qué celular tiene la última versión de android?",

    # Intención "celulares_por_marca"
    "Samsung",
    "Apple",
    "Xiaomi",
    "Motorola",
    "Realme",
    "Huawei",
    "Google Pixel",
    "OnePlus",
    "Asus",
    "Sony",
    "¿Qué celulares tiene Samsung?",

    # Intención "pantalla_y_calidad"
    "pantalla",
    "AMOLED",
    "Hercios",
    "Hz",
    "pulgadas",
    "OLED",
    "HDR10",
    "resolución",
    "¿Qué celular tiene buena pantalla?",

    # Intención "celulares_nuevos_y_antiguos"
    "nuevo",
    "este año",
    "reciente",
    "última generación",
    "¿Qué celulares nuevos hay?",
   ]


# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas if respuesta is not None]

    if y_true and y_pred:
        y_true_encoded = label_encoder.transform(y_true)
        y_pred_encoded = label_encoder.transform(y_pred)

        accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
        precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
        recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
        f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

        print(f"Precisión (Accuracy): {accuracy:.2f}")
        print(f"Precisión (Precision): {precision:.2f}")
    else:
        print("No se pudieron generar predicciones para calcular las métricas.")


preguntas_evaluacion = [
    # Intención "saludo"
    "Hola",
    "¿Cómo estás?",
    "Buenos días",
    "¡Qué tal!",
    "Hey",
    "¿Qué onda?",
    "Dame saludos",
    "¿Me saludas?",

    # Intención "despedida"
    "Adiós",
    "Hasta luego",
    "Nos vemos",
    "Chao",
    "Me voy",
    "Bye",
    "Hasta la vista",
    "Cuídate",
    "Debo irme",
    "Despídete",

    # Intención "agradecimientos"
    "gracias",
    "muchas gracias",
    "mil gracias",
    "muy amable",
    "se lo agradezco",
    "fue de ayuda",
    "gracias por la ayuda",
    "muy agradecido",
    "gracias por tu tiempo",
    "gracias por responder",
    "Te agradezco mucho",

    # Intención "celulares_para_juegos"
    "jugar",
    "gaming",
    "potente para juegos",
    "rendimiento en juegos",
    "Teléfonos gaming",
    "Me gustan los juegos",
    "para jugar",
    "buena GPU",
    "mejor celular gamer",
    "¿Qué celular es bueno para jugar?",

    # Intención "cámara_y_fotografia"
    "buena cámara",
    "tomar fotos",
    "cámara de alta calidad",
    "mejor cámara",
    "fotografía",
    "fotos profesionales",
    "buen zoom",
    "cámara nocturna",
    "estabilización óptica",
    "cámara frontal",
    "¿Qué celular tiene buena cámara para fotos?",

    # Intención "resistencia_y_durabilidad"
    "resistente",
    "no se rompa",
    "¡Hola! ¿En qué puedo ayudarte hoy?",
    "¡Saludos! ¿Qué necesitas?",
    "¡Hey! ¿Cómo estás?",
    "¡Qué gusto verte!",


    # Intención "despedida"
    "¡Hasta luego! Que tengas un buen día.",
    "¡Adiós! ¡Vuelve pronto!",
    "¡Nos vemos! No dudes en preguntar cuando lo necesites.",
    "¡Cuídate!",
    "Bye, hasta la próxima.",

    # Intención "agradecimientos"
    "De nada, estoy aquí para ayudar.",
    "Feliz por ayudar.",
    "Gracias a ti por preguntar.",
    "¡Siempre a la orden!",
    "Fue un placer ayudarte.",

    # Intención "celulares_para_juegos"
    "Para gaming, te recomiendo un celular con un buen procesador y al menos 8GB de RAM.",
    "Los teléfonos con Snapdragon 8 Gen 2 y buena tasa de refresco son ideales para juegos.",
    "Busca un celular con una GPU potente y refrigeración eficiente.",
    "Algunos celulares gaming incluyen botones físicos y triggers.",
    "¿Tienes un presupuesto en mente? Puedo recomendarte algo según tu rango de precio.",
    # Intención "cámara_y_fotografía"
    "Para fotografía, te sugiero un celular con al menos 50MP en la cámara principal y buen procesamiento de imagen.",
    "Los celulares de gama alta suelen tener mejores cámaras, pero hay opciones más accesibles con buenas prestaciones.",
    "Busca un celular con estabilización óptica para mejores fotos en movimiento.",
    "Algunos modelos cuentan con teleobjetivo para capturas a larga distancia.",
    "¿Prefieres un celular con cámara profesional o algo más equilibrado?",

    # Intención "resistencia_y_durabilidad"
    "Si buscas un celular resistente, te recomiendo modelos con certificación IP68 o con protección militar.",
                "Algunos celulares como los CAT o Samsung XCover están diseñados para resistir golpes y polvo.",
                "Los celulares con Gorilla Glass suelen tener pantallas más resistentes.",
                "Si trabajas en ambientes exigentes, te sugiero un smartphone todoterreno.",
    # Intención "trabajo_y_productividad"
    "Si buscas un celular para trabajo, te sugiero modelos con buena batería y almacenamiento amplio.",
                "Algunos modelos premium incluyen funciones específicas para productividad como Samsung Dex o Apple Continuity.",
                "Si necesitas tomar notas, te recomiendo un celular con lápiz óptico como la serie Galaxy Note o S Ultra.",
                "¿Buscas algo potente o más enfocado en batería y conectividad?",

    # Intención "software_y_actualizaciones"
    "Si buscas un celular con actualizaciones garantizadas, los Google Pixel y Samsung tienen buen soporte.",
                "Apple ofrece soporte de software por más años que la mayoría de marcas de Android.",
                "Si te preocupa la seguridad, busca celulares con parches de seguridad mensuales.",
                "Los celulares con Android puro suelen recibir actualizaciones más rápido que los personalizados.",

    # Intención "celulares_por_marca"
    "Claro, aquí tienes modelos de la marca que buscas.",
                "Estos son algunos de los mejores celulares de esa marca.",
                "Déjame filtrar los modelos disponibles de esa marca.",
                "Los celulares de esta marca tienen diferentes opciones, ¿buscas algo en específico?",
                "Puedo mostrarte los últimos lanzamientos o modelos populares de esa marca.",

    # Intención "pantalla_y_calidad"
    "Si buscas una pantalla grande, te recomendaría un modelo con más de 6.5 pulgadas.",
                "Las pantallas AMOLED ofrecen mejor contraste y colores más vivos.",
                "Los celulares con 120Hz ofrecen una experiencia más fluida al navegar y jugar.",
                "Si quieres la mejor calidad de pantalla, busca modelos con HDR10 o Dolby Vision.",
                "¿Te interesa más la calidad de imagen o el tamaño de la pantalla?",

    # Intención "celulares_nuevos_y_antiguos"
    "Aquí tienes una lista de los celulares más recientes.",
                "Estos son algunos modelos lanzados en los últimos meses.",
                "Si buscas lo último en tecnología, estos celulares pueden interesarte.",
                "¿Te gustaría un modelo de gama alta o prefieres algo más asequible?",
                "Déjame buscar los modelos más recientes en nuestra base de datos.",
 ]
evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas)

Precisión (Accuracy): 0.56
Precisión (Precision): 0.52


BOT3.0 con error en las respuestas (supongo que es debido a que no logra reconocer el .json)


IntentosXEficiencia.json ( es el documento generado)

In [41]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "IntentosXEficiencia.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas]

    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

    print(f"Precisión (Accuracy): {accuracy:.2f}")
    print(f"Precisión (Precision): {precision:.2f}")

# Ejemplo de preguntas y respuestas para la evaluación
preguntas_evaluacion = [
    "¿Qué tipo de pantalla tiene el iPhone 13?",
    "¿Cuánto cuesta el Samsung Galaxy S22?",
    "¿Qué procesador tiene el Xiaomi Redmi Note 11?",
    "¿Cuántos megapíxeles tiene la cámara del Google Pixel 6?",
    "¿Tiene el OnePlus 10 Pro carga rápida?",
    "¿Qué versión de Android tiene el Samsung Galaxy A53?",
    "¿Es resistente al agua el iPhone 13 Pro?",
    "¿Tiene el Xiaomi 12 lector de huellas digitales?",
    "¿Qué celulares tienen 5G?",
    "¿Qué celulares tienen buena cámara para selfies?",
    "¿Cuál es el mejor celular para juegos?",
    "¿Qué celular tiene la mejor batería?",
    "¿Qué celular tiene la mejor cámara?",
    "¿Qué celular tiene la mejor pantalla?",
    "¿Qué celular es el más resistente?",
    "¿Qué celular tiene la mejor relación calidad-precio?",
    "¿Qué celular tiene la mejor conectividad?",
    "¿Qué celular tiene la mejor seguridad?",
    "¿Qué celular tiene la mejor experiencia de usuario?",
    "¿Qué celular tiene el mejor diseño?",
    "¿Qué celulares tienen pantalla AMOLED?",
    "¿Qué celulares tienen 120Hz?",
    "¿Qué celulares tienen carga inalámbrica?",
    "¿Qué celulares tienen zoom óptico?",
    "¿Qué celulares tienen altavoces estéreo?",
    "¿Qué celulares tienen jack de auriculares?",
    "¿Qué celulares tienen ranura para tarjeta microSD?",
    "¿Qué celulares tienen lápiz óptico?",
    "¿Qué celulares tienen pantalla plegable?",
    "¿Qué celulares tienen pantalla curva?",
    "¿Qué celulares tienen notch?",
    "¿Qué celulares tienen agujero en pantalla?",
    "¿Qué celulares tienen cámara frontal debajo de la pantalla?",
    "¿Qué celulares tienen cámara TOF?",
    "¿Qué celulares tienen sensor LiDAR?",
    "¿Qué celulares tienen sensor de ritmo cardíaco?",
    "¿Qué celulares tienen sensor de temperatura?",
    "¿Qué celulares tienen sensor de luz ultravioleta?",
    "¿Qué celulares tienen sensor de huellas dactilares ultrasónico?",
    "¿Qué celulares tienen sensor de huellas dactilares óptico?"
]

respuestas_esperadas = [
    "El iPhone 13 tiene una pantalla OLED Super Retina XDR de 6.1 pulgadas.",
    "El precio del Samsung Galaxy S22 varía según la configuración, pero ronda los $799.",
    "El Xiaomi Redmi Note 11 tiene un procesador Qualcomm Snapdragon 680.",
    "La cámara principal del Google Pixel 6 tiene 50 megapíxeles.",
    "Sí, el OnePlus 10 Pro tiene carga rápida SuperVOOC de 80W.",
    "El Samsung Galaxy A53 viene con Android 12.",
    "Sí, el iPhone 13 Pro es resistente al agua con certificación IP68.",
    "Sí, el Xiaomi 12 tiene lector de huellas digitales en pantalla.",
    "Algunos celulares con 5G son: iPhone 13, Samsung Galaxy S22, Xiaomi 12, Google Pixel 6, OnePlus 10 Pro.",
    "Algunos celulares con buena cámara para selfies son: Google Pixel 6, iPhone 13, Samsung Galaxy S22.",
    "El mejor celular para juegos depende de tus preferencias, pero algunos modelos populares son el Asus ROG Phone 6, el RedMagic 7S Pro y el iPhone 14 Pro Max.",
    "El celular con la mejor batería depende de tus necesidades, pero algunos modelos con buena autonomía son el Asus ROG Phone 6, el Motorola Edge+ y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor cámara depende de tus preferencias, pero algunos modelos con cámaras destacadas son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor pantalla depende de tus preferencias, pero algunos modelos con pantallas excelentes son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "El celular más resistente depende de tus necesidades, pero algunos modelos con buena resistencia son el CAT S62 Pro, el Samsung Galaxy XCover 6 Pro y el Doogee S100.",
    "El celular con la mejor relación calidad-precio depende de tus necesidades y presupuesto, pero algunos modelos populares son el Google Pixel 6a, el Samsung Galaxy A53 y el Xiaomi Redmi Note 11 Pro.",
    "El celular con la mejor conectividad depende de tus necesidades, pero algunos modelos con buena conectividad son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "El celular con la mejor seguridad depende de tus necesidades, pero algunos modelos con buena seguridad son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con la mejor experiencia de usuario depende de tus preferencias, pero algunos modelos con buena experiencia de usuario son el Google Pixel 7 Pro, el iPhone 14 Pro Max y el Samsung Galaxy S23 Ultra.",
    "El celular con el mejor diseño depende de tus preferencias, pero algunos modelos con diseños atractivos son el Samsung Galaxy S23 Ultra, el iPhone 14 Pro Max y el Google Pixel 7 Pro.",
    "Algunos celulares con pantalla AMOLED son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11.",
    "Algunos celulares con 120Hz son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con carga inalámbrica son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con zoom óptico son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, Xiaomi 13 Pro.",
    "Algunos celulares con altavoces estéreo son: Samsung Galaxy S23 Ultra, iPhone 14 Pro Max, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con jack de auriculares son: Asus ROG Phone 6, Sony Xperia 1 IV, algunos modelos de gama de entrada y media.",
    "Algunos celulares con ranura para tarjeta microSD son: Algunos modelos de Samsung Galaxy A, algunos modelos de Xiaomi Redmi Note.",
    "Algunos celulares con lápiz óptico son: Samsung Galaxy S23 Ultra, Samsung Galaxy Note 20 Ultra.",
    "Algunos celulares con pantalla plegable son: Samsung Galaxy Z Fold 4, Samsung Galaxy Z Flip 4, Motorola Razr 2022.",
    "Algunos celulares con pantalla curva son: Samsung Galaxy S23 Ultra, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con notch son: iPhone 13, iPhone 12, algunos modelos de gama de entrada y media.",
    "Algunos celulares con agujero en pantalla son: Samsung Galaxy S23 Ultra, Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro.",
    "Algunos celulares con cámara frontal debajo de la pantalla son: Samsung Galaxy Z Fold 4, Xiaomi Mix 4.",
    "Algunos celulares con cámara TOF son: Algunos modelos de gama alta de Huawei, algunos modelos de gama alta de Samsung.",
    "Algunos celulares con sensor LiDAR son: iPhone 14 Pro Max, iPhone 13 Pro Max, iPad Pro.",
    "Algunos celulares con sensor de ritmo cardíaco son: Algunos modelos de Samsung Galaxy Watch.",
    "Algunos celulares con sensor de temperatura son: Algunos modelos de Huawei Watch.",
    "Algunos celulares con sensor de luz ultravioleta son: Algunos modelos de Samsung Galaxy S.",
    "Algunos celulares con sensor de huellas dactilares ultrasónico son: Samsung Galaxy S23 Ultra, Samsung Galaxy S22 Ultra.",
    "Algunos celulares con sensor de huellas dactilares óptico son: Google Pixel 7 Pro, OnePlus 11, Xiaomi 13 Pro."
]
evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas)


while True:
    pregunta = input("👤 Escribe tu consulta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        break
    print("🤖", responder_chatbot(pregunta))



Precisión (Accuracy): 0.78
Precisión (Precision): 0.60
👤 Escribe tu consulta (o 'salir' para terminar): celular Samsung
🤖  I have an iphone 7 and I love it. It's the best phone I've ever owned.
👤 Escribe tu consulta (o 'salir' para terminar): en español
🤖  I'm not sure what you mean by that, but I do know that celery is a type of fruit.
👤 Escribe tu consulta (o 'salir' para terminar): ¿cual es el mejor telefono para gaming?
🤖  I'm not sure what you mean by that, but I do know that celery is a type of fruit.
👤 Escribe tu consulta (o 'salir' para terminar): salir


4.0 con parametros del intents.json

In [40]:
import json
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Métricas de evaluación

# Cargar modelos de Hugging Face
modelo_emb_name = "distilbert-base-uncased"
modelo_resp_name = "facebook/blenderbot-400M-distill"

tokenizer_emb = AutoTokenizer.from_pretrained(modelo_emb_name)
modelo_emb = AutoModel.from_pretrained(modelo_emb_name)

tokenizer_resp = AutoTokenizer.from_pretrained(modelo_resp_name)
modelo_resp = AutoModelForSeq2SeqLM.from_pretrained(modelo_resp_name)

# Cargar el dataset del chatbot (intents.json)
archivo_intents = "intents.json"
try:
    with open(archivo_intents, "r", encoding="utf-8") as file:
        intents = json.load(file)
except FileNotFoundError:
    intents = {"intents": []}

# Cargar dataset de celulares
archivo_celulares = "celulares_limpios.json"
try:
    with open(archivo_celulares, "r", encoding="utf-8") as file:
        celulares = json.load(file)
except FileNotFoundError:
    celulares = []

# Memoria de la conversación (para recordar contexto)
memoria_chat = []

def obtener_embedding(frase):
    tokens = tokenizer_emb(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        salida = modelo_emb(**tokens)
    return salida.last_hidden_state[:, 0, :].numpy().flatten()

patterns, labels = [], []

for intent in intents["intents"]:
    if "tag" in intent and "patterns" in intent:
        for pattern in intent["patterns"]:
            patterns.append(pattern)
            labels.append(intent["tag"])

if patterns:
    X = np.array([obtener_embedding(frase) for frase in patterns])
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(labels)

    classifier = SVC(kernel="linear", probability=True)
    classifier.fit(X, y)
else:
    classifier = None

def predecir_intencion(frase):
    if classifier:
        embedding = obtener_embedding(frase)
        prediccion = classifier.predict([embedding])[0]
        return label_encoder.inverse_transform([prediccion])[0]
    return None

def obtener_respuesta_json(intencion):
    for intent in intents["intents"]:
        if intent["tag"] == intencion:
            return np.random.choice(intent["responses"])
    return None

def generar_respuesta_transformer(frase):
    tokens = tokenizer_resp(frase, return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        respuesta_ids = modelo_resp.generate(**tokens)
    return tokenizer_resp.decode(respuesta_ids[0], skip_special_tokens=True)

def respuesta_valida(respuesta):
    palabras_invalidas = ["no sé", "no entiendo", "no puedo ayudar", "no tengo información"]
    return not any(palabra in respuesta.lower() for palabra in palabras_invalidas)

def guardar_aprendizaje(nueva_pregunta, nueva_respuesta):
    global intents

    if not respuesta_valida(nueva_respuesta):
        return

    intencion_predicha = predecir_intencion(nueva_pregunta)

    if intencion_predicha:
        for intent in intents["intents"]:
            if intent["tag"] == intencion_predicha:
                if nueva_pregunta not in intent["patterns"]:
                    intent["patterns"].append(nueva_pregunta)
                if nueva_respuesta not in intent["responses"]:
                    intent["responses"].append(nueva_respuesta)
                break
    else:
        nueva_categoria = f"aprendizaje_{len(intents['intents'])+1}"
        intents["intents"].append({
            "tag": nueva_categoria,
            "patterns": [nueva_pregunta],
            "responses": [nueva_respuesta]
        })

    with open(archivo_intents, "w", encoding="utf-8") as f:
        json.dump(intents, f, indent=4, ensure_ascii=False)

def manejar_contexto(texto):
    global memoria_chat
    memoria_chat.append(texto)
    if len(memoria_chat) > 3:
        memoria_chat.pop(0)
    return " ".join(memoria_chat)

def responder_chatbot(texto):
    texto_con_contexto = manejar_contexto(texto)
    intencion = predecir_intencion(texto_con_contexto)
    respuesta = obtener_respuesta_json(intencion)

    if respuesta:
        return respuesta

    for celular in celulares:
        if any(palabra in texto.lower() for palabra in [celular["Company Name"].lower(), celular["Model Name"].lower()]):
            return json.dumps(celular, indent=4)

    respuesta_generada = generar_respuesta_transformer(texto_con_contexto)

    if respuesta_valida(respuesta_generada):
        guardar_aprendizaje(texto, respuesta_generada)

    return respuesta_generada

# Función para evaluar el chatbot
def evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas):
    respuestas_predichas = [responder_chatbot(pregunta) for pregunta in preguntas_evaluacion]

    # Convertir las etiquetas de texto a numéricas para la evaluación
    y_true = [predecir_intencion(pregunta) for pregunta in preguntas_evaluacion]
    y_pred = [predecir_intencion(respuesta) for respuesta in respuestas_predichas]

    y_true_encoded = label_encoder.transform(y_true)
    y_pred_encoded = label_encoder.transform(y_pred)

    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted', zero_division=0)

    print(f"Precisión (Accuracy): {accuracy:.2f}")
    print(f"Precisión (Precision): {precision:.2f}")

# Ejemplo de preguntas y respuestas para la evaluación

preguntas_evaluacion = [
    # Intención "saludo"
    "Hola",
    "¿Cómo estás?",
    "Buenos días",
    "¡Qué tal!",
    "Hey",
    "¿Qué onda?",
    "Dame saludos",
    "¿Me saludas?",

    # Intención "despedida"
    "Adiós",
    "Hasta luego",
    "Nos vemos",
    "Chao",
    "Me voy",
    "Bye",
    "Hasta la vista",
    "Cuídate",
    "Debo irme",
    "Despídete",

    # Intención "agradecimientos"
    "gracias",
    "muchas gracias",
    "mil gracias",
    "muy amable",
    "se lo agradezco",
    "fue de ayuda",
    "gracias por la ayuda",
    "muy agradecido",
    "gracias por tu tiempo",
    "gracias por responder",
    "Te agradezco mucho",

    # Intención "celulares_para_juegos"
    "jugar",
    "gaming",
    "potente para juegos",
    "rendimiento en juegos",
    "Teléfonos gaming",
    "Me gustan los juegos",
    "para jugar",
    "buena GPU",
    "mejor celular gamer",
    "¿Qué celular es bueno para jugar?",

    # Intención "cámara_y_fotografia"
    "buena cámara",
    "tomar fotos",
    "cámara de alta calidad",
    "mejor cámara",
    "fotografía",
    "fotos profesionales",
    "buen zoom",
    "cámara nocturna",
    "estabilización óptica",
    "cámara frontal",
    "¿Qué celular tiene buena cámara para fotos?",

    # Intención "resistencia_y_durabilidad"
    "resistente",
    "no se rompa",
    "¡Hola! ¿En qué puedo ayudarte hoy?",
    "¡Saludos! ¿Qué necesitas?",
    "¡Hey! ¿Cómo estás?",
    "¡Qué gusto verte!",


    # Intención "despedida"
    "¡Hasta luego! Que tengas un buen día.",
    "¡Adiós! ¡Vuelve pronto!",
    "¡Nos vemos! No dudes en preguntar cuando lo necesites.",
    "¡Cuídate!",
    "Bye, hasta la próxima.",

    # Intención "agradecimientos"
    "De nada, estoy aquí para ayudar.",
    "Feliz por ayudar.",
    "Gracias a ti por preguntar.",
    "¡Siempre a la orden!",
    "Fue un placer ayudarte.",

    # Intención "celulares_para_juegos"
    "Para gaming, te recomiendo un celular con un buen procesador y al menos 8GB de RAM.",
    "Los teléfonos con Snapdragon 8 Gen 2 y buena tasa de refresco son ideales para juegos.",
    "Busca un celular con una GPU potente y refrigeración eficiente.",
    "Algunos celulares gaming incluyen botones físicos y triggers.",
    "¿Tienes un presupuesto en mente? Puedo recomendarte algo según tu rango de precio.",
    # Intención "cámara_y_fotografía"
    "Para fotografía, te sugiero un celular con al menos 50MP en la cámara principal y buen procesamiento de imagen.",
    "Los celulares de gama alta suelen tener mejores cámaras, pero hay opciones más accesibles con buenas prestaciones.",
    "Busca un celular con estabilización óptica para mejores fotos en movimiento.",
    "Algunos modelos cuentan con teleobjetivo para capturas a larga distancia.",
    "¿Prefieres un celular con cámara profesional o algo más equilibrado?",

    # Intención "resistencia_y_durabilidad"
    "Si buscas un celular resistente, te recomiendo modelos con certificación IP68 o con protección militar.",
                "Algunos celulares como los CAT o Samsung XCover están diseñados para resistir golpes y polvo.",
                "Los celulares con Gorilla Glass suelen tener pantallas más resistentes.",
                "Si trabajas en ambientes exigentes, te sugiero un smartphone todoterreno.",
    # Intención "trabajo_y_productividad"
    "Si buscas un celular para trabajo, te sugiero modelos con buena batería y almacenamiento amplio.",
                "Algunos modelos premium incluyen funciones específicas para productividad como Samsung Dex o Apple Continuity.",
                "Si necesitas tomar notas, te recomiendo un celular con lápiz óptico como la serie Galaxy Note o S Ultra.",
                "¿Buscas algo potente o más enfocado en batería y conectividad?",

    # Intención "software_y_actualizaciones"
    "Si buscas un celular con actualizaciones garantizadas, los Google Pixel y Samsung tienen buen soporte.",
                "Apple ofrece soporte de software por más años que la mayoría de marcas de Android.",
                "Si te preocupa la seguridad, busca celulares con parches de seguridad mensuales.",
                "Los celulares con Android puro suelen recibir actualizaciones más rápido que los personalizados.",

    # Intención "celulares_por_marca"
    "Claro, aquí tienes modelos de la marca que buscas.",
                "Estos son algunos de los mejores celulares de esa marca.",
                "Déjame filtrar los modelos disponibles de esa marca.",
                "Los celulares de esta marca tienen diferentes opciones, ¿buscas algo en específico?",
                "Puedo mostrarte los últimos lanzamientos o modelos populares de esa marca.",

    # Intención "pantalla_y_calidad"
    "Si buscas una pantalla grande, te recomendaría un modelo con más de 6.5 pulgadas.",
                "Las pantallas AMOLED ofrecen mejor contraste y colores más vivos.",
                "Los celulares con 120Hz ofrecen una experiencia más fluida al navegar y jugar.",
                "Si quieres la mejor calidad de pantalla, busca modelos con HDR10 o Dolby Vision.",
                "¿Te interesa más la calidad de imagen o el tamaño de la pantalla?",

    # Intención "celulares_nuevos_y_antiguos"
    "Aquí tienes una lista de los celulares más recientes.",
                "Estos son algunos modelos lanzados en los últimos meses.",
                "Si buscas lo último en tecnología, estos celulares pueden interesarte.",
                "¿Te gustaría un modelo de gama alta o prefieres algo más asequible?",
                "Déjame buscar los modelos más recientes en nuestra base de datos.",
 ]
evaluar_chatbot(preguntas_evaluacion, respuestas_esperadas)

Precisión (Accuracy): 0.57
Precisión (Precision): 0.52
